In [1]:
#bert中文情感模型预训练过程
import torch#导入 PyTorch，一个流行的深度学习框架，用于构建和训练神经网络。
from datasets import load_dataset#从 datasets 库中引入 load_dataset 方法，这个库可以方便地加载各种预定义的数据集
from transformers import AdamW#初始化优化器和损失函数
from transformers import BertModel
from transformers import BertTokenizer

In [4]:
# 设置设备，检查是否有 GPU 可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


#定义数据集
class Dataset(torch.utils.data.Dataset):#定义一个名为 Dataset 的类，继承自 torch.utils.data.Dataset，这是 PyTorch 中的一个基础类，用于创建自定义数据集。
    def __init__(self, split):#初始化函数，在创建类的实例时调用。
        #使用 datasets 库的 load_dataset 方法加载名为 ChnSentiCorp 的中文情感分类数据集
        self.dataset = load_dataset('parquet',data_files={split:f'./ChnSentiCorp/{split}.parquet'})
        self.split = split
    def __len__(self):#定义 __len__ 方法，返回数据集的样本数。
        return len(self.dataset[self.split])
    def __getitem__(self, i):#定义 __getitem__ 方法，用于通过索引 i 获取单个样本。
        text = self.dataset[self.split][i]['text']
        label = self.dataset[self.split][i]['label']

        return text, label#返回文本和标签，作为数据集的单个样本。


dataset = Dataset('train')#创建 Dataset 类的实例，指定 split='train'，加载训练集数据。
token = BertTokenizer.from_pretrained('./bert-base-chinese')

#定义collate_fn对模型数据进行批处理
def collate_fn(data):
    # 从 data 中提取出每个样本的文本（sents）和标签（labels）
    sents = [i[0] for i in data]
    labels = [i[1] for i in data]

    
    #编码
    #使用 token.batch_encode_plus 来批量编码文本。这个函数来自于 Hugging Face 的 transformers 库。
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,#batch_text_or_text_pairs=sents：要编码的文本列表。
                                   truncation=True,#如果文本超过 max_length，则截断到指定的最大长度。
                                   padding='max_length',#将文本填充到 max_length 指定的长度。
                                   max_length=500,#指定最大长度为 500。
                                   return_tensors='pt',#返回 PyTorch 张量格式。
                                   return_length=True)#返回每个文本的实际长度。

    

    # 提取输入 id、attention mask 和 token type id
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    token_type_ids = data['token_type_ids'].to(device)
    labels = torch.LongTensor(labels).to(device)


    return input_ids, attention_mask, token_type_ids, labels


#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

#加载bert预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese')
pretrained = pretrained.to(device)

#不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)



#定义下游任务模型，继承自 PyTorch 的 torch.nn.Module。该模型使用了预训练的 BERT 模型（pretrained），并添加了一个全连接层（fc）作为分类头。
#定义模型类 Model
class Model(torch.nn.Module):#这是一个继承自 torch.nn.Module 的自定义类，用于定义下游任务模型。
    #是 PyTorch 中所有神经网络模型的基类，所有自定义模型都应该继承自它。
    def __init__(self):
        super().__init__()#super().__init__() 调用了父类（torch.nn.Module）的构造函数，确保模型正确初始化。
        self.fc = torch.nn.Linear(768, 2)#定义了一个全连接层 fc，它的输入维度是 768，输出维度是 2。

    
    #2. 定义前向传播（forward）方法
    def forward(self, input_ids, attention_mask, token_type_ids):#这是模型的前向传播方法，用于执行推理或训练过程中的计算。在前向传播中，数据将通过模型进行计算，并返回输出结果。
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,#是输入的文本（经过编码后的 token IDs）。
                       attention_mask=attention_mask,#是用于指示哪些位置为填充的掩码（0 表示填充，1 表示有效 token）
                       token_type_ids=token_type_ids)#是用于区分句子的标识符

        # 提取 [CLS] token 进行分类
        cls_embedding = out.last_hidden_state[:, 0]  # 形状: (batch_size, hidden_size)
        
        # 将输出移动到正确的设备上
        cls_embedding = cls_embedding.to(input_ids.device)  # 确保输出在与输入相同的设备上
        
        out = self.fc(cls_embedding)

        #应用 Softmax 激活函数
        out = out.softmax(dim=1)



        return out

Using device: cuda


Generating train split: 0 examples [00:00, ? examples/s]

F:\Adconna\envs\mypytorch1\lib\site-packages\transformers\modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_locatio

In [35]:
#测试
def test():
    model.eval()
    
    #初始化正确预测和总样本数
    correct = 0#初始化用于计数预测正确的样本数
    total = 0#初始化用于计数所有样本的总数。

    #创建测试数据加载器
    loader_test = torch.utils.data.DataLoader(dataset=Dataset('validation'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)
    #开始遍历测试集
    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):  

        #遍历5次测试集进行模型测试
        if i == 5:
            break
            
        #这个上下文管理器用于禁用梯度计算，表示我们在测试阶段只需要执行前向传播，并不需要计算梯度（不需要反向传播），这能节省内存和计算资源。
        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
        
        #计算预测结果并累积正确率
        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)
        accuracy = correct / total

    return accuracy




#创建和使用模型实例转移到gpu上运行
model = Model().to(device)

#训练
#这里我们使用了 AdamW 优化器，它是基于 Adam 优化器的变体，加入了权重衰减（weight decay）机制，通常用来训练带有预训练模型的任务（如 BERT）。
optimizer = AdamW(model.parameters(), lr=5e-4)


#指定了 交叉熵损失函数（CrossEntropyLoss），它是分类任务中常用的损失函数。对于多分类任务，交叉熵损失函数比较适合。
criterion = torch.nn.CrossEntropyLoss()

# 模型训练过程
model.train()#切换模型到训练模式。这是一个 PyTorch 的方法，它启用了 Dropout 和 Batch Normalization 等训练时的特殊行为。训练时和推理时（model.eval()）的行为是不同的。
after_large_test_accuracy = 0#初始化临时存放的评估准确率
num=0#准确率评估次数

#这是一个批量迭代过程，loader 是训练数据的 DataLoader。每次迭代中，从 loader 中获取一批数据，包括：
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):


    #前向传播（计算模型输出）
    out = model(input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

    #计算损失并进行反向传播
    loss = criterion(out, labels)#criterion(out, labels)：CrossEntropyLoss 会自动对 out 应用 softmax（将 logits 转化为概率），然后计算损失。
    loss.backward()#loss.backward()：进行 反向传播，计算损失函数相对于模型参数的梯度。这个步骤会通过链式法则计算每个模型参数的梯度。
    optimizer.step()#optimizer.step()：使用优化器根据计算得到的梯度来更新模型的参数。即通过梯度下降算法调整权重，使损失函数最小化。
    optimizer.zero_grad()#optimizer.zero_grad()：在每次更新参数之前，清除之前计算的梯度。这是因为 PyTorch 中的梯度是累加的，因此每次梯度更新后都需要手动清空之前的梯度，否则它们会相加，影响下一次更新。
   
    if i % 5 == 0:#每经过 5 次迭代（即每 5 个 batch）打印一次训练信息，包括当前的损失和准确率。
        out = out.argmax(dim=1)#out.argmax(dim=1)：argmax(dim=1) 会返回 out 中每个样本的最大值索引，这个索引对应的是模型的预测类别。dim=1 表示沿类别维度进行最大值索引操作。
        
        accuracy = (out == labels).sum().item() / len(labels)#accuracy = (out == labels).sum().item() / len(labels)：计算当前批次的 准确率。通过比较预测结果（out）与实际标签（labels），得出预测正确的样本数，然后除以批次的总样本数，得到准确率。

        temp_test_accuracy = test()
        front_large_test_accuracy = after_large_test_accuracy#保存准确率最大时的数据
        #每一次训练模型后进行模型评估,采用提取终止策略,时刻监视测试集上的准确率，保存测试集上准确率最大时刻的模型，提高模型泛化性能        
        if temp_test_accuracy>after_large_test_accuracy:
            num = 0
            after_large_test_accuracy = temp_test_accuracy 
            #torch.save(model.state_dict(), 'sentiment_model.pth')  
            # 保存整个模型
            torch.save(model, './model/sentiment_model_full.pth')
        elif front_large_test_accuracy == after_large_test_accuracy:
             num +=1
             if num>10:
               break  

        #打印当前的迭代步数（i）、损失值（loss.item()）和准确率，当前时刻测试集准确率、测试集最大准确率持续次数
        print(i, loss.item(), accuracy,temp_test_accuracy,front_large_test_accuracy,num)


   #终止条件（训练最多进行 301 次迭代）
    if i == 300:
        break
        #if i == 300:：当迭代次数达到 300 时，手动 终止训练。这意味着最多进行 301 个批次的训练（从 0 开始）。

0 0.6908183097839355 0.5625 0.4625 0 0
5 0.6885359883308411 0.625 0.56875 0.4625 0
10 0.6132141351699829 0.8125 0.74375 0.56875 0
15 0.6094036102294922 0.625 0.71875 0.74375 1
20 0.6087853908538818 0.75 0.80625 0.74375 0
25 0.6029424071311951 0.8125 0.775 0.80625 1
30 0.566361129283905 0.875 0.775 0.80625 2
35 0.573114275932312 0.8125 0.8625 0.80625 0
40 0.5539938807487488 0.8125 0.80625 0.8625 1
45 0.5200412273406982 0.9375 0.85625 0.8625 2
50 0.5252194404602051 0.8125 0.85625 0.8625 3
55 0.5761885643005371 0.8125 0.78125 0.8625 4
60 0.5383076667785645 0.8125 0.875 0.8625 0
65 0.44062837958335876 1.0 0.8625 0.875 1
70 0.4612167477607727 0.9375 0.7875 0.875 2
75 0.4838995933532715 0.875 0.875 0.875 3
80 0.5190228819847107 0.8125 0.825 0.875 4
85 0.5019806623458862 0.9375 0.88125 0.875 0
90 0.6404908895492554 0.625 0.85 0.88125 1
95 0.45633289217948914 0.875 0.83125 0.88125 2
100 0.5419474840164185 0.875 0.83125 0.88125 3
105 0.434012234210968 0.9375 0.8625 0.88125 4
110 0.4209659397602

In [5]:
#基于训练好的模型对评论进行测试
# 加载整个模型
model = torch.load('./model/sentiment_model_full.pth')
model.eval()  # 切换到评估模式

C:\Users\11952\AppData\Local\Temp\ipykernel_22144\3974306987.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('./model/sentiment_model_full.pth')


Model(
  (fc): Linear(in_features=768, out_features=2, bias=True)
)

In [6]:
from transformers import BertTokenizer

# 初始化分词器
tokenizer = BertTokenizer.from_pretrained('./bert-base-chinese')
def preprocess_text(text, max_length=128):
    # 对文本进行编码，转换为 BERT 所需的输入格式
    encoding = tokenizer(text, 
                         truncation=True, 
                         padding='max_length', 
                         max_length=max_length, 
                         return_tensors='pt')
    return encoding

# 示例文本
text = "这部电影真的很棒！"

# 预处理文本
encoding = preprocess_text(text)
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)

In [7]:
# 进行情感预测
with torch.no_grad():
    outputs = model(input_ids=input_ids, 
                    attention_mask=attention_mask, 
                    token_type_ids=None)  # 如果使用句子对任务，可以设置 token_type_ids

# 获取预测的类别，假设是二分类任务，0表示负面，1表示正面
pred = torch.argmax(outputs, dim=1)

# 打印预测结果
if pred.item() == 1:
    print("情感分析结果：正面")
else:
    print("情感分析结果：负面")

情感分析结果：正面


In [10]:
# 假设有多个文本进行批量预测
texts = ["这是一座晚上最出片的宝藏寺庙。大佛古寺位于广州北京路商圈中心，夜晚的大佛寺尤为亮眼，琉璃金光笼罩着整个寺庙，金碧辉煌，气势辉煌，在灯光的映照下，可谓现实版的千与千寻非常适合拍照，随便一个角落都能拍出美美的照片。庙内的每尊佛像都十分精美壮观。哈喽。！", "这部电影非常糟糕。"]

# 预处理所有文本
encodings = [preprocess_text(text) for text in texts]
input_ids_batch = torch.cat([encoding['input_ids'] for encoding in encodings], dim=0).to(device)
attention_mask_batch = torch.cat([encoding['attention_mask'] for encoding in encodings], dim=0).to(device)

# 进行情感预测
with torch.no_grad():
    outputs = model(input_ids=input_ids_batch, 
                    attention_mask=attention_mask_batch, 
                    token_type_ids=None)

# 获取每个文本的预测结果
predictions = torch.argmax(outputs, dim=1)

# 打印预测结果
for text, pred in zip(texts, predictions):
    sentiment = "正面" if pred.item() == 1 else "负面"
    print(f"文本: {text} -> 情感分析结果: {sentiment}")


文本: 这是一座晚上最出片的宝藏寺庙。大佛古寺位于广州北京路商圈中心，夜晚的大佛寺尤为亮眼，琉璃金光笼罩着整个寺庙，金碧辉煌，气势辉煌，在灯光的映照下，可谓现实版的千与千寻非常适合拍照，随便一个角落都能拍出美美的照片。庙内的每尊佛像都十分精美壮观。哈喽。！ -> 情感分析结果: 正面
文本: 这部电影非常糟糕。 -> 情感分析结果: 负面
